## Dependencies

In [ ]:
!pip install newspaper3k beautifulsoup4 readability-lxml tiktoken mistralai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 33.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install SPARQLWrapper networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.1 MB/s eta 0:00:00


In [ ]:
import requests
import time
import concurrent.futures
import http.client
import json
from json import JSONDecodeError
import math

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import tiktoken

def count_tokens(text: str) -> int:
  encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
  return len(encoding.encode(text))

In [ ]:
from google.colab import userdata
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

mistral_api_key = userdata.get('MISTRAL_API_KEY')
mistral_client = MistralClient(api_key=mistral_api_key)

In [ ]:
from openai import OpenAI

oai_api_key = userdata.get('OPENAI_KEY')
oai_client = OpenAI(api_key=oai_api_key)

## Keyword Generation

Generates a list of keywords from natural language query submitted by the user

In [ ]:
from google.colab import userdata

MAX_ANSWER_TOKENS = 13000

# KEYWORD_SYSTEM_PROMPT = '''
# <s>
# [INST]
# You are an artificial intelligence model created by Devflow Studio experienced in natural language processing. You are tasked with helping your user perform better searches through a search engine. The user will provide you a raw query that you need to polish and rewrite to get best search results from a search engine. Given a user query, you are tasked with generating a set of keywords suitable for a search engine query, with the added capability to discern and specify the type of search required — be it a web search, a place search, or a news search — based on the query's context and intent. This feature, along with enhancing capabilities for image search, accommodating local searches, and generating an entity for the Knowledge Graph based on WikiData, forms a comprehensive approach to query processing. The response should be structured as a JSON object containing the following keys based on the query's context and requirements:

# 1. **`queries`**: An array of 1 to 3 keywords encapsulating the main topics or questions within the query, prioritizing brevity and relevance. Breakdown the question into parts to get best results. For simple and straightforward questions, you can simply create a single query in an array. For more complex queries that might require extensive research, break down each question from the query into simpler parts. Make sure to use your training knowledge to write keywords that yield best results from a search engine.

# 2. **`type`**: Determine whether the query should lead to a `websearch`, `place`, or `news` search based on the user's intent and the nature of the query. This type classification guides the search engine on how to best fulfill the query.

# 3. **`image_search`**: (Optional, defaults to false) Use image search if the query's context suggests that visual results would be beneficial. You use image search when you believe that visual aid may help answer user's query better or might result in more engagement. Image search shall be used for queries about people, places / landmarks, objects (things), products, etc.

# 4. **`entity`**: (Optional) When the main subject of the query is about a person, place, or organization/company, include this field with the name of the entity as recognized by Wikipedia for Knowledge Graph generation. Knowledge Graphs are rendered in the frontend with a visually appealing UI. The Knowledge Graph will be help the user learn in an engaging way rather than reading just text.

# **Guidelines for the Model:**

# - **Determining the Search Type:**
#    - **Web Search (`websearch`)**: Default for queries seeking general information, explanations, guides, or knowledge not tied to a specific location or recent events.
#    - **Place Search (`place`)**: For queries specifically asking for local services, businesses, restaurants, hotels or attractions, indicating the need for localized information.
#    - **News Search (`news`)**: When the query is about current events, updates, or recent developments, necessitating the latest news coverage.

# - **When to Search for Images:** Initiate an image search for queries where visuals significantly enhance understanding or engagement, focusing on people, places/landmarks, products, etc.

# - **Utilizing User Location:** Use the user's location for "near me" requests or when local context is vital, adapting keywords to include local specificity.

# - **Identifying Entities for Knowledge Graph:** Focus on accurately identifying entities for people, places, or organizations. For ambiguous references or multiple entities, prioritize based on query context and use disambiguation strategies. The entity must be recognised by Wikipedia to be able to generate a Knowledge Graph for it. Make sure to write a complete and proper entity name. For example: when user wants to know about manufacturer of iPhone, "Apple" will result in the fruit whereas "Apple Inc" will result in the Cupertino-based company.

# Strictly adhere to responding in JSON format, ensuring clarity, brevity, and adherence to the outlined format.
# Try your best at the given task. Even if there's info missing, try your best to build a query out of it. You are not allowed to ask question. You are only tasked with responding to the user and not backforth communication.

# Here's an example:
# User Query: "Where are some places I can visit nearby"
# User Location: "Seattle, Washington"
# [/INST]
# {"queries": ["tourist spots seattle washington"], "type": "place", "image_search": true, "entity": "Seattle"}
# [INST]
# Here's another example:
# User Query: "Who is the CEO of Tesla"
# [/INST]
# {"queries": ["Tesla CEO"], "type": "websearch", "image_search": true}
# [INST]
# Here's another example:
# User Query: "Who is Sam Altman"
# [/INST]
# {"queries": ["Sam Altman", "Sam Altman biography"], "type": "websearch", "image_search": true, "entity": "Sam Altman"}
# </s>
# '''

# SEARCH_BUILDER_MESSAGES = [
#   ('system', KEYWORD_SYSTEM_PROMPT),
#   ('user', 'User Query: "How does photosynthesis work?"'),
#   ('assistant', '{"queries": ["photosynthesis process"], "type": "websearch"}'),
#   ('user', 'User Query: "Best Italian restaurants near me"\nUser Location: San Francisco'),
#   ('assistant', '{"queries": ["Italian restaurants San Francisco"], "type": "place"}'),
#   ('user', 'User Query: "Latest updates on elections"\nUser Location: New Delhi'),
#   ('assistant', '{"queries": ["election news new delhi", "election new india"], "type": "news"}'),
#   ('user', 'User Query: "Show me pictures of the Taj Mahal at sunrise"'),
#   ('assistant', '{"queries": ["Taj Mahal sunrise pictures"], "image": {"queries": ["Taj Mahal sunrise"]}, "entity": "Taj Mahal", "type": "websearch"}'),
#   ('user', 'User Query: "What are the best places to visit near me?"\nUser Location: New York'),
#   ('assistant', '{"queries": ["tourist spots new york"], "type": "place"}')
# ]

# def build_search(query: str, location: str | None = None, attempt: int = 1) -> dict:
#   try:
#     start_time = time.time()

#     query = f"User: '{query}'"
#     if location:
#       query += f"\nUser Location: {location}"

#     messages = []

#     for role, content in SEARCH_BUILDER_MESSAGES:
#       messages.append(ChatMessage(role=role, content=content))

#     user_message = f'User Query: "{query}"'
#     if location:
#       user_message += f'\nUserLocation: {location}'

#     messages.append(ChatMessage(role="user", content=user_message))

#     response = mistral_client.chat(
#       model="open-mistral-7b",
#       messages=messages,
#     )

#     print(f"Generated keywords in {time.time() - start_time} seconds")

#     return json.loads(response.choices[0].message.content)
#   except Exception as e:
#     if attempt > 1:
#       print(f'Attempt {attempt} resulted in error {e}')
#     else:
#       print(f'Error occured building search info {e}')
#     if attempt < 5:
#       return build_search(query, location, attempt + 1)
#     else:
#       return None

KEYWORD_SYSTEM_PROMPT = '''
You are an artificial intelligence model created by Devflow Studio experienced in natural language processing. You are tasked with helping your user perform better searches through a search engine. The user will provide you a raw query that you need to polish and rewrite to get best search results from a search engine. Given a user query, you are tasked with generating a set of keywords suitable for a search engine query, with the added capability to discern and specify the type of search required — be it a web search, a place search, or a news search — based on the query's context and intent. This feature, along with enhancing capabilities for image search, accommodating local searches, and generating an entity for the Knowledge Graph based on WikiData, forms a comprehensive approach to query processing. The response should be structured as a JSON object containing the following keys based on the query's context and requirements:

1. **`queries`**: An array of 1 to 3 keywords encapsulating the main topics or questions within the query, prioritizing brevity and relevance. Breakdown the question into parts to get best results. For simple and straightforward questions, you can simply create a single query in an array. For more complex queries that might require extensive research, break down each question from the query into simpler parts. Make sure to use your training knowledge to write keywords that yield best results from a search engine.

2. **`type`**: Determine whether the query should lead to a `websearch`, `place`, or `news` search based on the user's intent and the nature of the query. This type classification guides the search engine on how to best fulfill the query.

3. **`image_search`**: (Optional, defaults to false) Use image search if the query's context suggests that visual results would be beneficial. You use image search when you believe that visual aid may help answer user's query better or might result in more engagement. Image search shall be used for queries about people, places / landmarks, objects (things), products, etc.

4. **`entity`**: (Optional) When the main subject of the query is about a person, place, or organization/company, include this field with the name of the entity as recognized by Wikipedia for Knowledge Graph generation. Knowledge Graphs are rendered in the frontend with a visually appealing UI. The Knowledge Graph will be help the user learn in an engaging way rather than reading just text.

**Guidelines for the Model:**

- **Determining the Search Type:**
   - **Web Search (`websearch`)**: Default for queries seeking general information, explanations, guides, or knowledge not tied to a specific location or recent events.
   - **Place Search (`place`)**: For queries specifically asking for local services, businesses, restaurants, hotels or attractions, indicating the need for localized information.
   - **News Search (`news`)**: When the query is about current events, updates, or recent developments, necessitating the latest news coverage.

- **When to Search for Images:** Initiate an image search for queries where visuals significantly enhance understanding or engagement, focusing on people, places/landmarks, products, etc.

- **Utilizing User Location:** Use the user's location for "near me" requests or when local context is vital, adapting keywords to include local specificity.

- **Identifying Entities for Knowledge Graph:** Focus on accurately identifying entities for people, places, or organizations. For ambiguous references or multiple entities, prioritize based on query context and use disambiguation strategies. The entity must be recognised by Wikipedia to be able to generate a Knowledge Graph for it. Make sure to write a complete and proper entity name. For example: when user wants to know about manufacturer of iPhone, "Apple" will result in the fruit whereas "Apple Inc" will result in the Cupertino-based company.

Strictly adhere to responding in JSON format, ensuring clarity, brevity, and adherence to the outlined format.
Try your best at the given task. Even if there's info missing, try your best to build a query out of it. You are not allowed to ask question. You are only tasked with responding to the user and not backforth communication.

Here are a few examples:

User Query: "Where are some places I can visit nearby"
User Location: "Seattle, Washington"
Expected Function Call: {"keywords": ["tourist spots seattle washington"], "search_type": "places", "image_search": true, "entity": "Seattle"}

User Query: "Who is the CEO of Tesla"
Expected Function Call: {"keywords": ["Tesla CEO"], "search_type": "websearch", "image_search": true}

User Query: "Who is Sam Altman"
Expected Function Call: {"keywords": ["Sam Altman", "Sam Altman biography"], "search_type": "websearch", "image_search": true, "entity": "Sam Altman"}

User Query: "How does photosynthesis work?"
Expected Function Call: {"keywords": ["photosynthesis process"], "search_type": "websearch"}

User Query: "Best Italian restaurants near me"
User Location: San Francisco
Expected Function Call: {"keywords": ["Italian restaurants San Francisco"], "search_type": "places"}

User Query: "Latest updates on elections"
User Location: New Delhi
Expected Function Call: {"keywords": ["election news new delhi", "election new india"], "search_type": "news"}

User Query: "Show me pictures of the Taj Mahal at sunrise"
Expected Function Call: {"keywords": ["Taj Mahal sunrise pictures"], "image_search": true, "entity": "Taj Mahal", "search_type": "websearch"}

User Query: "What are the best places to visit near me?"
User Location: New York
Expected Function Call: {"keywords": ["tourist spots new york"], "image_search": true, "search_type": "place"}
'''

def build_search(query: str, location: str | None = None, attempt: int = 1) -> dict:
  try:
    tool = {
      "type": "function",
      "function": {
        "name": "build_search_info",
        "description": "Generates search info for search engine",
        "parameters": {
          "type": "object",
          "properties": {
            "keywords": {
              "type": "array",
              "description": "An array of 1 to 3 keywords encapsulating the main topics or questions within the query, prioritizing brevity and relevance. Breakdown the question into parts to get best results. For simple and straightforward questions, you can simply create a single query in an array. For more complex queries that might require extensive research, break down each question from the query into simpler parts. Make sure to use your training knowledge to write keywords that yield best results from a search engine.",
              "items": {
                "type": "string"
              }
            },
            "search_type": {
              "type": "string",
              "enum": ["websearch", "news", "places"],
              "description": "Determine whether the query should lead to a `websearch`, `places`, or `news` search based on the user's intent and the nature of the query. This type classification guides the search engine on how to best fulfill the query."
            },
            "image_search": {
              "type": "boolean",
              "description": "Use image search if the query's context suggests that visual results would be beneficial. You use image search when you believe that visual aid may help answer user's query better or might result in more engagement. Image search shall be used for queries about people, places / landmarks, objects (things), products, etc."
            },
            "entity": {
              "type": "string",
              "description": "When the main subject of the query is about a person, place, or organization/company, include this field with the name of the entity as recognized by Wikipedia for Knowledge Graph generation. Knowledge Graphs are rendered in the frontend with a visually appealing UI. The Knowledge Graph will be help the user learn in an engaging way rather than reading just text."
            }
          },
          "required": ["queries", "search_type"],
        },
      },
    }

    user_message = f'User Query: "{query}"'
    if location:
      user_message += f'\nUserLocation: {location}'

    response = oai_client.chat.completions.create(
      model="gpt-3.5-turbo-0125",
      messages=[
        {
          "role": "system",
          "content": KEYWORD_SYSTEM_PROMPT
        },
        {
          "role": "user",
          "content": user_message
        },
      ],
      tools=[tool],
      tool_choice={"type": "function", "function": {"name": "build_search_info"}}
    )

    func = json.loads(response.choices[0].message.tool_calls[0].function.arguments)

    return func
  except Exception as e:
    if attempt > 1:
      print(f'Attempt {attempt} resulted in error {e}')
    else:
      print(f'Error occured building search info {e}')
    if attempt < 5:
      return build_search(query, location, attempt + 1)
    else:
      return None

In [ ]:
build_search("Tourist spots nearby", location="Shimla")

Error occured building search info name 'OPENAI_KEYWORD_SYSTEM_PROMPT' is not defined
Attempt 2 resulted in error name 'OPENAI_KEYWORD_SYSTEM_PROMPT' is not defined
Attempt 3 resulted in error name 'OPENAI_KEYWORD_SYSTEM_PROMPT' is not defined
Attempt 4 resulted in error name 'OPENAI_KEYWORD_SYSTEM_PROMPT' is not defined
Attempt 5 resulted in error name 'OPENAI_KEYWORD_SYSTEM_PROMPT' is not defined


## Knowledge Graph

Build knowledge graphs for website and mobile app that display the info in an engaging way similar to Google's Knowledge Panels.

Use-cases:

1. People
2. Places
3. Business Profiles (Places, Organisations)

In [ ]:
def clean_dict(input_dict: dict) -> dict:
  """
  Removes keys from a dictionary where the corresponding value is None.

  :param input_dict: Dictionary to be cleaned
  :return: A new dictionary with all keys with None values removed
  """
  # Create a copy of the keys list to safely modify the dict while iterating
  for key in list(input_dict.keys()):
    # Remove the key if the value is None or an empty string
    if type(input_dict[key]) is list and len(input_dict[key]) == 1:
      input_dict[key] = input_dict[key][0]
    if not input_dict[key] or input_dict[key] == "":
      del input_dict[key]
  return input_dict

def fix_links(graph: dict) -> dict:
  '''
  Replaces the Facebook, Instagram and Twitter and LinkedIn usernames with their links

  :param graph: Graph generated by the function
  :return: Graph modified by replacing the usernames with links
  '''

  social_networks = {
    'facebook': 'https://www.facebook.com/{}',
    'instagram': 'https://instagram.com/{}',
    'twitter': 'https://twitter.com/{}',
    'linkedin': 'https://linkedin.com/in/{}',
  }

  graph['social_links'] = {}

  for network, url_format in social_networks.items():
    if network in graph:
      username = graph[network]
      if type(username) is list:
        username = username[0]
      graph['social_links'][network] = url_format.format(username)
      del graph[network]

  return graph

In [ ]:
def search_entity(entity: str, language: str = "en"):
  """
  Search for a Wikidata entity by query and return its Q number.

  Parameters:
  query (str): The search query to find the entity on Wikidata.

  Returns:
  str: The Q number of the first matching entity, or None if no match is found.
  """

  wikidata_search_url = "https://www.wikidata.org/w/api.php"
  params = {
    "action": "wbsearchentities",
    "language": language,
    "format": "json",
    "search": entity
  }
  try:
    response = requests.get(wikidata_search_url, params=params)
    response.raise_for_status()  # Raises HTTPError for bad responses

    # Parse the JSON response
    data = response.json()

    # Check if search results exist, if yes then return the first result
    if data['search']:
      return data['search'][0]
    else:
      return None
  except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
    return None
  except requests.exceptions.RequestException as err:
    print(f"Request exception: {err}")
    return None

# Example usage
# query = "openai"
# entity = search_entity(query)
# q_number = entity['id']
# label = entity['display']['label']['value']
# description = entity['display']['description']['value']
# print(f"Q number for '{query}': {q_number}")
# print(label)
# print(description)

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import networkx as nx

def get_knowledge_graph(q_number, language: str = "en"):
  try:
    # Configure the SPARQL query endpoint
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

    # SPARQL query to fetch properties and values for the given person
    query = f"""
    SELECT ?property ?propertyLabel ?value ?valueLabel
    WHERE {{
      wd:{q_number} ?property ?value .
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "{language}". }}
    }}
    """

    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    if len(results['results']['bindings']) <= 0:
      return None

    property_labels = {
      'P18': 'image',
      'P2002': 'twitter',
      'P2013': 'facebook',
      'P2003': 'instagram',
      'P6634': 'linkedin',
      'P856': 'website',
    }

    attribute_labels = {
      'P106': 'Occupation',              # Person
      'P101': 'Field of Work',           # Person
      'P800': 'Notable Work',            # Person / Organisation
      'P27': 'Country of Citizenship',   # Person
      'P27': 'Nationality',              # Person
      'P569': 'Date of Birth',           # Person
      'P22': 'Father',                   # Person
      'P25': 'Mother',                   # Person
      'P3373': 'Siblings',               # Person
      'P26': 'Spouse',                   # Person
      'P40': 'Children',                 # Person
      'P2218': 'Net Worth',              # Person
      'P108': 'Employer',                # Person
      'P39': 'Position Held',            # Person
      'P2048': 'Height',                 # Person
      'P21': 'Gender',                   # Person
      'P102': 'Political Affiliation',   # Person
      'P69': 'Educated at',              # Person

      'P131': 'City',                    # Place
      'P1376': 'Capital Of',             # Place
      'P2046': 'Area',                   # Place
      'P1082': 'Population',             # Place
      'P36': 'Capital',                  # Place
      'P6': 'Head of Government',        # Place
      'P571': 'Inception',               # Place

      'P101': 'Field of Work',           # Organisation (Company)
      'P169': 'CEO',                     # Organisation (Company)
      'P452': 'Industry',                # Organisation (Company)
      'P159': 'Headquarters',            # Organisation (Company)
      'P112': 'Founded by',              # Organisation (Company) / Place
      'P3320': 'Board Members',          # Organisation (Company)
      'P17': 'Country',                  # Organisation (Company) / Place
      'P1128': 'Employees',              # Organisation (Company)

      'P166': 'Award Received',          # Person / Organisation / Place
      'P1830': 'Owner of',               # Person / Organisation
    }

    # Initialize a dictionary to store the extracted information
    graph = {label: [] for label in property_labels.values()}
    attributes = {attribute: [] for attribute in attribute_labels.values()}

    # Process the SPARQL result
    for binding in results['results']['bindings']:
      prop = binding['property']['value'].split('/')[-1]  # Extract property ID from URI
      value_label = binding.get('valueLabel', {}).get('value', None)
      if value_label.startswith("statement/"):
        value_label = None
      if (prop in property_labels or prop in attribute_labels) and value_label:
        readable_label = property_labels.get(prop) or attribute_labels.get(prop)
        if prop in property_labels:
          graph[readable_label].append(value_label)
        elif prop in attribute_labels:
          attributes[readable_label].append(value_label)

    graph = clean_dict(graph)
    graph = fix_links(graph)
    graph['attributes'] = clean_dict(attributes)

    return graph
  except Exception as e:
    print(f"Problem encountered {e}")
    return None

# graph = get_knowledge_graph(q_number)

# for key, value in graph.items():
#   print(f"{key}: {value}\n")

In [ ]:
import time

def generate_knowledge_panel(query: str, language: str = "en") -> dict:
  """
  Generates a Knowledge Panel/Graph for the given entity (query)

  Parameters:
  query (str): The search query to find the entity on Wikidata.

  Returns:
  dict: A dictionary of knowledge graph with image, label, description and attributes
  """
  start_time = time.time()
  entity = search_entity(query, language)

  if not entity:
    return None

  q_number = entity['id']
  label = entity['display']['label']['value']
  description = entity['display']['description']['value']

  graph = get_knowledge_graph(q_number, language)

  if not graph:
    return None

  graph['label'] = label
  graph['description'] = description

  print(f"Generated Knowledge Panel in {time.time() - start_time} seconds")

  return graph

In [ ]:
generate_knowledge_panel('Seattle')

Generated Knowledge Panel in 0.9962115287780762 seconds


{'image': 'http://commons.wikimedia.org/wiki/Special:FilePath/Seattle%20aerial%202%2C%20May%202023.png',
 'website': 'https://www.seattle.gov/',
 'social_links': {'twitter': 'https://twitter.com/CityofSeattle'},
 'attributes': {'City': 'King County',
  'Capital Of': 'King County',
  'Area': ['369.243614', '369.466202'],
  'Population': '737015',
  'Head of Government': 'Bruce Harrell',
  'Inception': '1851-11-13T00:00:00Z',
  'Country': 'United States of America',
  'Owner of': ['Climate Pledge Arena',
   'Seattle City Hall',
   'Seattle Center Monorail',
   'First Hill Streetcar (Seattle)',
   'Seattle Channel',
   'South Lake Union Streetcar']},
 'label': 'Seattle',
 'description': 'city in and county seat of King County, Washington, United States'}

## SERPER API

In [ ]:
serper_api_key = userdata.get('SERPER_API_KEY')

def _serper_results(queries: list[str], search_type: str, n: int = 10):
  """
  Performs a batch of serper searches for each keyword in the given list.

  :param keywords: The list of keywords you would like to search
  :param n: The number of results required from the search
  :param search_type: websearch, image, or place

  :returns: A list of search results
  """

  url = "/search"
  results_store = "organic"

  if search_type == "websearch":
    url = "/search"
    results_store = "organic"
  elif search_type == "image":
    url = "/images"
    results_store = "images"
  elif search_type == "place":
    url = "/places"
    results_store = "places"
  elif search_type == "news":
    url = "/news"
    results_store = "news"

  try:
    start_time = time.time()
    conn = http.client.HTTPSConnection("google.serper.dev")
    payload = json.dumps([{"q": query} for query in queries])
    headers = {
      'X-API-KEY': serper_api_key,
      'Content-Type': 'application/json'
    }
    conn.request("POST", url, payload, headers)
    res = conn.getresponse()
    data = res.read()
    batch_result = json.loads(data.decode("utf-8"))

    results = []
    for result in batch_result:
      results += result[results_store]

    results = sorted(results, key=lambda x: x['position'])[:n]

    print(f"Total time taken to perform batch search: {time.time() - start_time} seconds")

    return results
  except Exception as e:
    print(f"Failed to perform a serper search {e}")
    return None

def web_search(queries: list[str], n: int = 10):
  """
  Performs a batch of web searches for each keyword in the given list.

  :param keywords: The list of keywords you would like to search
  :param n: The number of results required from the search

  :returns: A list of search results
  """
  try:
    results = _serper_results(queries, "websearch", n)

    return results
  except Exception as e:
    print(f"Failed to perform a web search {e}")
    return None

def image_search(queries: list[str], n: int = 10):
  """
  Performs a batch of image searches for each keyword in the given list.

  :param keywords: The list of keywords you would like to search
  :param n: The number of results required from the search

  :returns: A list of image search results
  """
  try:
    results = _serper_results(queries, "image", n)

    processed_results = []

    for result in results:
      processed_results.append({
        'image': {
          'url': result['imageUrl'],
          'height': result['imageHeight'],
          'width': result['imageWidth']
        },
        'title': result['title'],
        'link': result['link']
      })

    return processed_results
  except Exception as e:
    print(f"Failed to perform an image search {e}")
    return None

def place_search(queries: list[str], n: int = 10):
  """
  Performs a batch of places searches for each keyword in the given list.

  :param keywords: The list of keywords you would like to search
  :param n: The number of results required from the search

  :returns: A list of places search results
  """
  try:
    results = _serper_results(queries, "place", n)

    processed_results = []

    for result in results:
      processed_results.append({
        'title': result.get('title'),
        'website': result.get('website', None),
        'category': result.get('category'),
        'location': {
          'address': result.get('address'),
          'latitude': result.get('latitude'),
          'longitude': result.get('longitude')
        },
        'rating': {
          'stars': result.get('rating'),
          'count': result.get('ratingCount')
        } if 'rating' in result else None,
        'image': {
          'url': result.get('thumbnailUrl'),
        } if 'thumbnailUrl' in result else None,
      })

    return processed_results
  except Exception as e:
    print(f"Failed to perform a place search {e}")
    return None

def news_search(queries: list[str], n: int = 10):
  """
  Performs a batch of places news searches for each keyword in the given list.

  :param keywords: The list of keywords you would like to search
  :param n: The number of results required from the search

  :returns: A list of places search results
  """
  try:
    results = _serper_results(queries, "news", n)

    processed_results = []

    for result in results:
      processed_results.append({
        'title': result.get('title'),
        'link': result.get('link', None),
        'date': result.get('date'),
        'image': result.get('imageUrl'),
        'source': result.get('source')
      })

    return processed_results
  except Exception as e:
    print(f"Failed to perform a news search {e}")
    return None

## Crawler

In [ ]:
import requests
from bs4 import BeautifulSoup, Comment
import time
from newspaper import Article

import requests
from bs4 import BeautifulSoup, Comment
from threading import Thread
import time

class TimeoutException(Exception):
  pass

def thread_target(url, result_container):
  try:
    # Fetch the web page
    response = requests.get(url)
    html_content = response.text

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    for tag in soup(['script', 'style', 'header', 'footer', 'nav', 'aside']):
      tag.decompose()

    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
      comment.extract()

    for tag in soup.find_all(['div', 'section'], {'class': ['advertisement', 'promo', 'ad-banner']}):
      tag.decompose()

    clean_text = soup.get_text(separator=' ', strip=True)

    result_container.append(clean_text)
  except Exception as e:
    print(f"An error occurred: {e}")
    result_container.append(None)

def extract_relevant_text(url):
  start_time = time.time()

  # Container for the result (clean text)
  result_container = []

  # Create and start the thread
  thread = Thread(target=thread_target, args=(url, result_container))
  thread.start()

  # Wait for the thread to complete or timeout
  thread.join(timeout=2)

  # Check if the thread has finished execution
  if thread.is_alive():
    print("Operation timed out")
    return None
  else:
    # Calculate time taken
    end_time = time.time()
    print(f"Time taken to execute: {end_time - start_time} seconds")
    return result_container[0] if result_container else None

def get_website_content(url):
  try:
    start_time = time.time()  # Start timing

    # Create an Article object
    article = Article(url)

    # Download the article content
    article.download()

    # Parse the article
    article.parse()

    # Optionally, you can also perform natural language processing on the article
    # article.nlp()

    end_time = time.time()  # End timing
    execution_time = end_time - start_time
    print(f"Crawled page in: {execution_time} seconds")

    # Return the article's main text
    return article.text

  except Exception as e:
    print(f"An error occurred while fetching content from {url}: {e}")
    return None

def crawl(urls):
  result = {}

  start_time = time.time()  # Record the start time of the entire operation

  with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_url = {executor.submit(extract_relevant_text, url): url for url in urls}
    for future in concurrent.futures.as_completed(future_to_url):
      url = future_to_url[future]
      try:
        content = future.result()  # Get the content from the future
        if content:
          result[url] = content
      except Exception as e:
        print(f"An error occurred while fetching content from {url}: {e}")

  end_time = time.time()  # Record the end time of the entire operation
  total_time = end_time - start_time  # Calculate total execution time

  print(f"Total time taken to crawl {len(urls)} URLs: {total_time} seconds")
  return result

## Summarizer

In [ ]:
_web_search_summary_prompt = """
Objective: Summarize the provided search results into a coherent, concise response that directly answers the user's query. Your response should synthesize information from the given citations without repeating any words unnecessarily. Each piece of information you include from the citations must be followed by an appropriate citation marker to indicate its source.

Context: You are provided with contents from multiple sources as follows:

{context}

----

Instructions:

Read and Comprehend: Carefully read the content provided from each citation to understand the key points relevant to the user's query.

Synthesize Information: Extract essential information, facts, or insights from the provided citations that directly address the user's query. Avoid including general or irrelevant details.

Avoid Repetition: Do not repeat any words unnecessarily in your summary. Aim for varied vocabulary and succinct phrasing to convey the information clearly and efficiently.

Incorporate Citations: After each piece of information derived from the sources, include an inline citation in the format [citation:X], where X corresponds to the source number. This citation should directly follow the sentence it supports. All the citation for a sentence shall appear at the end of the same sentence. For sentences with multiple citations, you are allowed to put them next to each other: [citation:1][citation:2]
You can skip citations if there are no sources provided for the given context

Answer from Context Only: Your summary must be strictly based on the information provided in the given context. Do not infer or add information not present in the citations.

Clarity and Coherence: Ensure your summary is logical, coherent, and easy to understand. Structure your response to flow naturally, linking pieces of information in a way that forms a cohesive answer to the query.

Markdown Syntax Usage Instructions: In addition to summarizing the search results, you are encouraged to format your answer using Markdown syntax to enhance readability and structure. You are allowed to empasise, underline, italicise text, use single level headlines, create blockquotes, create code blocks and horizontal rules only.
For questions that can be answered in a few words or a single sentence, put that in a blockquote followed by a paragraph of explanation.

**DO NOT put a list of citations/sources at the end. All citations must be made after each sentence, if applicable**

Output Format:
Your response should be formatted as a continuous paragraph or a list of bullet points, whichever best suits the synthesis of information from the provided sources. Remember to include citation markers appropriately.

Example Query: "What are the health benefits of green tea?"

Your Task: Using the content from [[citation:1]], [[citation:2]], and others, summarize the health benefits of green tea, including specific benefits mentioned in the sources. Remember to use citation markers after each benefit that you list based on the sources.
"""

stop_words = [
  "<|im_end|>",
  "[End]",
  "[end]",
  "\nReferences:\n",
  "\nSources:\n",
  "End.",
]

def summarise(content: str, query: str):
  messages = [
    # ChatMessage(role="system", content=_web_search_summary_prompt.format(context=content)),
    # ChatMessage(role="user", content=query),
    {
      "role": "system",
      "content": _web_search_summary_prompt.format(context=content)
    },
    {
      "role": "user",
      "content": query
    },
  ]

  # response = mistral_client.chat_stream(
  response = oai_client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=messages,
    temperature=0.2,
    stream=True,
  )

  summary = ""

  for chunk in response:
    text = chunk.choices[0].delta.content
    if not text:
      continue
    summary += chunk.choices[0].delta.content
    print(chunk.choices[0].delta.content, end="")

  return summary

## Run

In [ ]:
def generate_crawled_content(result: dict) -> str:
  to_be_summarised = ""

  tokens_used = 0

  if result.get("sources") or result.get("news"):
    links = [source.get('link') for source in result.get('sources') or result.get("news")]
    crawl_result = list(crawl(links).values())
    for i, content in enumerate(crawl_result):
      c_context = f"---\n[[citation:{i+1}]] {content}\n\n"
      c_tokens = count_tokens(c_context)
      if c_tokens + tokens_used < MAX_ANSWER_TOKENS:
        to_be_summarised += c_context
        tokens_used += c_tokens

  if result.get("places"):
    for place in result['places']:
      for key, value in place.items():
        to_be_summarised += f"{key}: {value}"

  return to_be_summarised

In [ ]:
import concurrent.futures

def search(query: str):
  start_time = time.time()

  result = {}

  def execute_search_type():
    return 'sources', web_search([query])

  def execute_image_search():
    return 'images', image_search([query])

  with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit each process as a separate task
    futures = [
      executor.submit(execute_search_type),
      executor.submit(execute_image_search),
    ]

    for future in concurrent.futures.as_completed(futures):
      result_type, data = future.result()
      if result_type:
        result[result_type] = data

  to_be_summarised = generate_crawled_content(result)

  print(f"Time to first token {time.time() - start_time} seconds")

  # if search_type != "places":
  summary = summarise(to_be_summarised, query)
  result['summary'] = summary

  return result

def search_pro(query: str, location: str | None = None):
  start_time = time.time()
  search_info = build_search(query, location)

  keywords = search_info['keywords']
  search_type = search_info['search_type']
  search_images = search_info.get("image_search", False)
  entity = search_info.get('entity')

  result = {}

  def execute_search_type():
    match search_type:
      case 'websearch':
        return 'sources', web_search(keywords)
      case 'places':
        return 'places', place_search(keywords)
      case 'news':
        return 'news', news_search(keywords)

  def execute_image_search():
    if search_images:
      return 'images', image_search(keywords)
    return None, None

  def execute_knowledge_panel():
    if entity:
      return 'knowledge-panel', generate_knowledge_panel(entity)
    return None, None

  with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit each process as a separate task
    futures = [
      executor.submit(execute_search_type),
      executor.submit(execute_image_search),
      executor.submit(execute_knowledge_panel)
    ]

    for future in concurrent.futures.as_completed(futures):
      result_type, data = future.result()
      if result_type:
        result[result_type] = data

  to_be_summarised = generate_crawled_content(result)

  print(f"Time to first token {time.time() - start_time} seconds")

  # if search_type != "places":
  summary = summarise(to_be_summarised, query)
  result['summary'] = summary

  return result

In [ ]:
query = "What’s the story of Aaron Swartz"

result = search(query)

Total time taken to perform batch search: 0.7133047580718994 seconds
Total time taken to perform batch search: 0.7772061824798584 seconds
Time taken to execute: 0.5608060359954834 seconds
Time taken to execute: 0.5964798927307129 seconds
Time taken to execute: 0.3380565643310547 seconds
Time taken to execute: 0.6799483299255371 seconds
Time taken to execute: 1.3836181163787842 seconds
Time taken to execute: 1.412717342376709 seconds
Time taken to execute: 1.4436779022216797 seconds
Time taken to execute: 0.9375176429748535 seconds
Time taken to execute: 1.9414324760437012 seconds
Total time taken to crawl 9 URLs: 1.94382643699646 seconds
Time to first token 2.768836736679077 seconds
Aaron Swartz was an American computer programmer, writer, political organizer, and Internet activist. He was known for his involvement in developing RSS and Reddit. Swartz was a passionate advocate for open access to information and fought against internet censorship, notably leading a campaign against the 

In [ ]:
result.get('knowledge-panel')
# result.get('places')

In [ ]:
result.keys()

dict_keys(['images', 'knowledge-panel', 'sources', 'summary'])